In [1]:
## importing libraries

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3 as sql
import io
import requests
import time
import datetime as dt


In [2]:
# Loading JHU COVID data into pandas df

urlbase = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'
file_n = 'time_series_covid19_confirmed_US.csv'

jhu = requests.get(urlbase + file_n)

jhu_cont = jhu.content

jh_df = pd.read_csv(io.StringIO(jhu_cont.decode('utf-8'))) 

jh_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/15/22,12/16/22,12/17/22,12/18/22,12/19/22,12/20/22,12/21/22,12/22/22,12/23/22,12/24/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,18847,18847,18847,18847,18847,18847,18961,18961,18961,18961
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,67221,67221,67221,67221,67221,67221,67496,67496,67496,67496
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7007,7007,7007,7007,7007,7007,7027,7027,7027,7027
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,7668,7668,7668,7668,7668,7668,7692,7692,7692,7692
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,17648,17648,17648,17648,17648,17648,17731,17731,17731,17731


In [3]:
# removing non-VA states from dataset
jh_df = jh_df.loc[jh_df['Province_State'] == 'Virginia']

# cleaning JHU dataset

# jh_df['FIPS'].isnull().sum() # running this line confirms no null values in the FIPS column.
# jh_df.isnull().sum().sum() # running this line shows no null values anywhere in the jh_df
jh_df = jh_df.astype({'FIPS': np.uint64}) # converting FIPS to int

# removing FIPS values not in Census data
jh_df.drop(jh_df.index[jh_df['FIPS'] == 80051], inplace=True)
jh_df.drop(jh_df.index[jh_df['FIPS'] == 90051], inplace=True)

# dropping redundant cols
jh_df.drop(['iso2','iso3','code3','Province_State','Country_Region','Combined_Key'], axis = 1, inplace = True)

jh_df.rename(columns = {'Admin2':'name'}, inplace = True)
jh_df.reset_index(inplace = True, drop = True)

jh_df

,UID,FIPS,name,Lat,Long_,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,12/15/22,12/16/22,12/17/22,12/18/22,12/19/22,12/20/22,12/21/22,12/22/22,12/23/22,12/24/22
0,84051001,51001,Accomack,37.767072,-75.632346,0,0,0,0,0,...,8554,8567,8567,8567,8578,8616,8629,8648,8665,8665
1,84051003,51003,Albemarle,38.020807,-78.554811,0,0,0,0,0,...,23986,24020,24020,24020,24070,24107,24139,24175,24187,24187
2,84051510,51510,Alexandria,38.814003,-77.081831,0,0,0,0,0,...,43478,43529,43529,43529,43657,43685,43765,43817,43880,43880
3,84051005,51005,Alleghany,37.786361,-80.002225,0,0,0,0,0,...,4247,4243,4243,4243,4250,4254,4258,4268,4273,4273
4,84051007,51007,Amelia,37.340810,-77.985846,0,0,0,0,0,...,3371,3374,3374,3374,3383,3384,3388,3395,3400,3400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,84051830,51830,Williamsburg,37.281313,-76.709051,0,0,0,0,0,...,2200,2200,2200,2200,2202,2202,2205,2207,2206,2206
129,84051840,51840,Winchester,39.170545,-78.173251,0,0,0,0,0,...,7883,7891,7891,7891,7903,7913,7920,7926,7932,7932
130,84051195,51195,Wise,36.974615,-82.624105,0,0,0,0,0,...,13285,13289,13289,13289,13300,13309,13327,13347,13358,13358
131,84051197,51197,Wythe,36.915820,-81.078341,0,0,0,0,0,...,9158,9168,9168,9168,9186,9193,9202,9214,9229,9229


In [4]:
# loading census CSV

census_df = pd.read_csv('Virginia_VA.csv')
census_df.head()

### one potential issue...multiple FIPS codes, some FIPS are further broken up into districts.
# solutions, remove any FIPS codes that repeat in Census CSV from both datasets, OR sum numerical cols.

C:\Users\Paul Song\AppData\Local\Temp\ipykernel_10256\4170310392.py:3: DtypeWarning: Columns (3,9,18,30,36,38,57,58,78,86,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  census_df = pd.read_csv('Virginia_VA.csv')


,FILEID,STUSAB,SUMLEV,GEOVAR,GEOCOMP,CHARITER,CIFSN_x,LOGRECNO,GEOID,GEOCODE,...,P0050001,P0050002,P0050003,P0050004,P0050005,P0050006,P0050007,P0050008,P0050009,P0050010
0,PLST,VA,50,0,0,0,0,2,0500000US51001,51001,...,416,277,122,0,155,0,139,45,19,75
1,PLST,VA,50,0,0,0,0,3,0500000US51003,51003,...,7481,1546,319,18,1201,8,5935,5709,0,226
2,PLST,VA,50,0,0,0,0,4,0500000US51005,51005,...,296,263,0,43,220,0,33,7,0,26
3,PLST,VA,50,0,0,0,0,5,0500000US51007,51007,...,103,83,0,0,83,0,20,0,0,20
4,PLST,VA,50,0,0,0,0,6,0500000US51009,51009,...,714,111,0,0,111,0,603,443,0,160


In [5]:
# cleaning census VA dataframe, removing unneccessary columns
# renaming columns appropriately

keep_cols = ['STUSAB', 'GEOID', 'GEOCODE', 'AREALAND', 'AREAWATR', 'BASENAME', 'NAME', 'POP100', 'HU100',
            'P0020002', 'P0020003', 'P0020004', 'P0020005', 'P0020006', 'P0020007', 'P0020008',
            'P0020009', 'P0020010', 'P0020011', 'P0020012', 'P0020013', 'P0020014', 'P0020015', 'P0020016',
            'P0020017', 'P0020018', 'P0020019', 'P0020020', 'P0020021', 'P0020022', 'P0020023',
            'P0020024', 'P0020025', 'P0020026', 'P0020027', 'H0010002', 'H0010003', 'P0050001',
            'P0050002', 'P0050003', 'P0050004', 'P0050005', 'P0050006', 'P0050007', 'P0050008', 'P0050009',
            'P0050010']
census_df.drop(columns = [col for col in census_df if col not in keep_cols], inplace = True)

census_df.rename({'STUSAB':'State', 'GEOCODE':'FIPS', 'POP100':'Total_Population', 'HU100':'Housing_Count',
            'P0020002':'Hispanic_Latino', 'P0020003':'Not_Hispanic_Latino', 'P0020004':'Single_Race_Pop', 'P0020005':'White', 
            'P0020006':'Black_African_American', 'P0020007':'American_Indian', 'P0020008':'Asian',
            'P0020009':'Pacific_Islander', 'P0020010':'Other_Race', 'P0020011':'Multiracial_Pop', 
            'P0020012':'Biracial_Pop', 'P0020013':'White_Black', 'P0020014':'White_American_Indian', 
            'P0020015':'White_Asian', 'P0020016':'White_Pacific_Islander',
            'P0020017':'White_Other', 'P0020018':'Black_American_Indian', 'P0020019':'Black_Asian', 
            'P0020020':'Black_Pacific_Islander', 'P0020021':'Black_Other', 'P0020022':'American_Indian_Asian', 
            'P0020023':'American_Indian_Pacific_Islander', 'P0020024':'American_Indian_Other', 
            'P0020025':'Asian_Pacific_Islander', 'P0020026':'Asian_Other', 'P0020027':'Pacific_Islander_Other', 
            'H0010002':'Occupied_Housing', 'H0010003':'Vacant_Housing', 'P0050001':'Group_Quarters_Population',
            'P0050002':'Institutionalized_Pop', 'P0050003':'Correctional_Pop', 'P0050004':'Juvenile_Correctional_Pop', 
            'P0050005':'Nursing_Home_Pop', 'P0050006':'Other_Institutional_Pop', 'P0050007':'Noninstitutionalized_Pop', 
            'P0050008':'College_Housing_Pop', 'P0050009':'Military_Quarters_Pop','P0050010':'Other_Noninstitutional_Pop'}, 
            axis = 1, inplace = True)

# row 241957 onwards in CSV starts listing schools, dropping those rows
census_df.drop(census_df.index[241955:], inplace = True)

## only keeping rows that have 5 digits in FIPS codes (longer FIPS code mean subdistricts, which is repeat data)
census_df['FIPS'] = census_df['FIPS'].astype(str) # converting to string
census_df.drop(census_df[census_df['FIPS'].map(len) > 5].index, inplace = True)
census_df['FIPS'] = census_df['FIPS'].astype('int64') # converting back to int

# dropping FIPS codes that are repeated, only keeping first instance
census_df.drop_duplicates(subset = ['FIPS'], keep = 'first', inplace=True)
census_df.reset_index(inplace = True, drop = True)

# converting population counts to float, so we can perform query operations like division
census_df[census_df.columns[7:].tolist()] = census_df[census_df.columns[7:].tolist()].astype('float64')

census_df


,State,GEOID,FIPS,AREALAND,AREAWATR,BASENAME,NAME,Total_Population,Housing_Count,Hispanic_Latino,...,Group_Quarters_Population,Institutionalized_Pop,Correctional_Pop,Juvenile_Correctional_Pop,Nursing_Home_Pop,Other_Institutional_Pop,Noninstitutionalized_Pop,College_Housing_Pop,Military_Quarters_Pop,Other_Noninstitutional_Pop
0,VA,0500000US51001,51001,1163725929,2229259960,Accomack,Accomack County,33413.0,21703.0,3430.0,...,416.0,277.0,122.0,0.0,155.0,0.0,139.0,45.0,19.0,75.0
1,VA,0500000US51003,51003,1866014147,14223772,Albemarle,Albemarle County,112395.0,47291.0,8453.0,...,7481.0,1546.0,319.0,18.0,1201.0,8.0,5935.0,5709.0,0.0,226.0
2,VA,0500000US51005,51005,1156616591,8162741,Alleghany,Alleghany County,15223.0,7934.0,178.0,...,296.0,263.0,0.0,43.0,220.0,0.0,33.0,7.0,0.0,26.0
3,VA,0500000US51007,51007,920379345,8188217,Amelia,Amelia County,13265.0,5704.0,425.0,...,103.0,83.0,0.0,0.0,83.0,0.0,20.0,0.0,0.0,20.0
4,VA,0500000US51009,51009,1227554631,12898694,Amherst,Amherst County,31307.0,14232.0,838.0,...,714.0,111.0,0.0,0.0,111.0,0.0,603.0,443.0,0.0,160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,VA,620L600US51092,51092,54030422,29180042,92,State House District 92,79095.0,36949.0,4742.0,...,2805.0,695.0,241.0,18.0,374.0,62.0,2110.0,2012.0,0.0,98.0
191,VA,620L600US51094,51094,87653645,91198197,94,State House District 94,83035.0,34841.0,8140.0,...,4844.0,259.0,78.0,11.0,170.0,0.0,4585.0,3368.0,1120.0,97.0
192,VA,620L600US51096,51096,507536857,327248105,96,State House District 96,92312.0,39319.0,5820.0,...,1821.0,871.0,0.0,0.0,582.0,289.0,950.0,0.0,807.0,143.0
193,VA,620L600US51098,51098,2735617837,940878748,98,State House District 98,79664.0,40781.0,2555.0,...,878.0,640.0,190.0,0.0,450.0,0.0,238.0,23.0,0.0,215.0


In [6]:
# dropping census data that is not in JHU dataset (i.e. dropping FIPS values not in JHU)

jhu_fips = jh_df['FIPS'].values.tolist() # creating list of JHU FIPS values
jhu_census = census_df[census_df['FIPS'].isin(jhu_fips)] # new df jhu_census only has census data with FIPS in JHU data

# dropping unneeded cols
jhu_census.drop(['State','BASENAME','NAME'], axis = 1, inplace = True)

# renaming cols to lower case
jhu_census.columns = jhu_census.columns.str.lower()

jhu_census

C:\Users\Paul Song\AppData\Local\Temp\ipykernel_10256\3206523351.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jhu_census.drop(['State','BASENAME','NAME'], axis = 1, inplace = True)


,geoid,fips,arealand,areawatr,total_population,housing_count,hispanic_latino,not_hispanic_latino,single_race_pop,white,...,group_quarters_population,institutionalized_pop,correctional_pop,juvenile_correctional_pop,nursing_home_pop,other_institutional_pop,noninstitutionalized_pop,college_housing_pop,military_quarters_pop,other_noninstitutional_pop
0,0500000US51001,51001,1163725929,2229259960,33413.0,21703.0,3430.0,29983.0,28878.0,19825.0,...,416.0,277.0,122.0,0.0,155.0,0.0,139.0,45.0,19.0,75.0
1,0500000US51003,51003,1866014147,14223772,112395.0,47291.0,8453.0,103942.0,99058.0,80335.0,...,7481.0,1546.0,319.0,18.0,1201.0,8.0,5935.0,5709.0,0.0,226.0
2,0500000US51005,51005,1156616591,8162741,15223.0,7934.0,178.0,15045.0,14542.0,13754.0,...,296.0,263.0,0.0,43.0,220.0,0.0,33.0,7.0,0.0,26.0
3,0500000US51007,51007,920379345,8188217,13265.0,5704.0,425.0,12840.0,12364.0,9687.0,...,103.0,83.0,0.0,0.0,83.0,0.0,20.0,0.0,0.0,20.0
4,0500000US51009,51009,1227554631,12898694,31307.0,14232.0,838.0,30469.0,28908.0,22967.0,...,714.0,111.0,0.0,0.0,111.0,0.0,603.0,443.0,0.0,160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0500000US51800,51800,1033819022,77045664,94324.0,38364.0,4252.0,90072.0,85569.0,43837.0,...,1585.0,1225.0,900.0,0.0,325.0,0.0,360.0,0.0,0.0,360.0
129,0500000US51810,51810,633828628,654687365,459470.0,190059.0,40404.0,419066.0,390359.0,269566.0,...,7286.0,2747.0,1212.0,71.0,1464.0,0.0,4539.0,905.0,2809.0,825.0
130,0500000US51820,51820,38771593,364644,22196.0,10268.0,1945.0,20251.0,19099.0,16074.0,...,276.0,243.0,0.0,4.0,239.0,0.0,33.0,0.0,0.0,33.0
131,0500000US51830,51830,23149770,416905,15425.0,5753.0,1215.0,14210.0,13613.0,7370.0,...,4873.0,115.0,0.0,0.0,115.0,0.0,4758.0,4755.0,0.0,3.0


In [7]:
# Separating positional columns from dated columns
jh_pos = jh_df[['UID','FIPS','name','Lat','Long_']]
jh_pos.columns = jh_pos.columns.str.lower()
jh_pos

,uid,fips,name,lat,long_
0,84051001,51001,Accomack,37.767072,-75.632346
1,84051003,51003,Albemarle,38.020807,-78.554811
2,84051510,51510,Alexandria,38.814003,-77.081831
3,84051005,51005,Alleghany,37.786361,-80.002225
4,84051007,51007,Amelia,37.340810,-77.985846
...,...,...,...,...,...
128,84051830,51830,Williamsburg,37.281313,-76.709051
129,84051840,51840,Winchester,39.170545,-78.173251
130,84051195,51195,Wise,36.974615,-82.624105
131,84051197,51197,Wythe,36.915820,-81.078341


In [8]:
# new df for daily cases + 'UID'
daily_cases = jh_df.columns[5:].tolist()
daily_jh = jh_df[['UID'] + daily_cases]


In [9]:
# pivoting with pd.melt()

daily_jh = pd.melt(frame = daily_jh, id_vars = ['UID'], var_name = 'case_date', value_name = 'num_cases')
daily_jh['case_date'] = pd.to_datetime(daily_jh['case_date'], format='%m/%d/%y')
daily_jh.rename(columns = {'UID':'uid'}, inplace = True) # lower case

daily_jh

,uid,case_date,num_cases
0,84051001,2020-01-22,0
1,84051003,2020-01-22,0
2,84051510,2020-01-22,0
3,84051005,2020-01-22,0
4,84051007,2020-01-22,0
...,...,...,...
142039,84051830,2022-12-24,2206
142040,84051840,2022-12-24,7932
142041,84051195,2022-12-24,13358
142042,84051197,2022-12-24,9229


In [10]:
# writing dataframes to a database

conn = sql.connect('covid-19.db')

#jhu_census = census data with releavant FIPS codes
#jh_df = cleaned jhu covid data with relavant FIPS
#jh_pos = jhu positional cols only
#daily_jh = UID and daily cases. Primary Key = UID + date

# census data
jhu_census.to_sql('census', conn, if_exists = 'replace', index = False)

# jhu location data
jh_pos.to_sql('locations', conn, if_exists='replace', index=False)

# jhu daily covid cases
daily_jh.to_sql('covid_cases', conn, if_exists='replace', index=False)

conn.close()